In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from Visual import animated_frames, save_frames
from Simulation import run
import numpy as np
from tqdm.autonotebook import tqdm

In [8]:
l = 1
d = l*2e-3
epsilon = d*1e-4

In [9]:
N = 500 #number of vertexes
Frames = 10 #number of frames between adding vercites
M = 50 #number of segment adding iteration between adding vertices
M = int(M/Frames)

In [10]:
N = 500 #number of vertexes
np.random.seed(3)
animation_segments,animation_vertices = run(N,M,Frames,l,d,epsilon,6)

/home/paulina/Politechnika/Inżynierka/Simulation.py:181: RuntimeWarning: invalid value encountered in double_scalars
  dist_x = dist_x*d/r
/home/paulina/Politechnika/Inżynierka/Simulation.py:182: RuntimeWarning: invalid value encountered in double_scalars
  dist_y = dist_y*d/r


In [11]:
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=667)

interactive(children=(Play(value=0, description='Press play', interval=200, max=667), Output()), _dom_classes=…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [7]:
save_frames(animation_segments, animation_vertices,"results_multi_vertices_3",3)

NameError: name 'animation_segments' is not defined

In [ ]:
!./nice_gif.sh results_multi_vertices_1/%06d.png animations/anim3.mp4 animations/anim3.gif

In [15]:
vertices = [np.array([1,1]), np.array([1,-1])]
N = len(vertices); F = 100; M = 10
l = 1; d = l*2e-3; epsilon = d*1e-4
output = run(N, M, F, l, d, epsilon, test=True, initial_vertices=vertices)
%matplotlib widget
animated_frames(output["animation_segments"], output["animation_vertices"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0)

interactive(children=(Play(value=0, description='Press play', interval=200), Output()), _dom_classes=('widget-…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [14]:
###TEST 6
vertices = [np.array([1,1]), np.array([1,-1]), np.array([0.5,0.5]), np.array([0.5,-0.5])]
N = len(vertices); F = 100; M = 10
l = 1; d = l*2e-3; epsilon = d*1e-4
output = run(N, M, F, l, d, epsilon, test=True, initial_vertices=vertices)
%matplotlib widget
animated_frames(output["animation_segments"], output["animation_vertices"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=101)

interactive(children=(Play(value=0, description='Press play', interval=200, max=101), Output()), _dom_classes=…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [32]:
###TEST 7
vertices = [np.array([1,1]), np.array([1,-1]), np.array([0.5,0.5])]
N = len(vertices); F = 100; M = 10
l = 1; d = l*2e-3; epsilon = d*1e-4
output = run(N, M, F, l, d, epsilon, test=True, initial_vertices=vertices)
%matplotlib widget
animated_frames(output["animation_segments"], output["animation_vertices"])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=94)

interactive(children=(Play(value=0, description='Press play', interval=200, max=94), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [31]:
vertices = [np.array([0.5,0]), np.array([0.2,0])]
segments = [(0,0),(0.75,0)]
N = len(vertices); F = 100; M = 10
l = 1; d = l*2e-3; epsilon = d*1e-4
output = run(N, M, F, l, d, epsilon, test=True, initial_vertices=vertices, initial_segments=segments)
print(len(output["segments"]))
%matplotlib widget
animated_frames(output["animation_segments"], output["animation_vertices"])


527


/home/paulina/Politechnika/Inżynierka/Simulation.py:181: RuntimeWarning: invalid value encountered in double_scalars
  dist_x = dist_x*d/r
/home/paulina/Politechnika/Inżynierka/Simulation.py:182: RuntimeWarning: invalid value encountered in double_scalars
  dist_y = dist_y*d/r


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=28)

interactive(children=(Play(value=0, description='Press play', interval=200, max=28), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>

In [3]:
from Simulation import reset, find_segments, segments_adding
segments, active_vertices, animation_vertices, animation_segments = reset()
active_vertices.append(np.array([0.5,0]))
segments.append((0.75,0))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
for f in range(150):
    L = len(segments)
    active_segments, segments_vertices = segments_adding(10,active_vertices,active_segments,segments_vertices,segments,2e-3,2e-7)
    if L != len(segments):
        animation_vertices.append(np.array(active_vertices))
        animation_segments.append(np.array(segments))
active_segments, segments_vertices = find_segments(active_vertices, segments,2e-3,2e-7)
animation_vertices.append(np.array(active_vertices))
animation_segments.append(np.array(segments))
segments = np.vstack(np.array(segments)).T
%matplotlib widget
animated_frames(animation_segments, animation_vertices)

False
True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

IntSlider(value=0, max=25)

interactive(children=(Play(value=0, description='Press play', interval=200, max=25), Output()), _dom_classes=(…

<function Visual.animated_frames.<locals>.update(f: int = 0)>